In [1]:
# !pip install chromadb
# !pip install chromadb==0.4.0
!pip install chromadb==0.5.0
# !pip install gentopia==0.0.4
# !pip install numpy==1.26.4
# !pip install openai==0.28.0
# !pip install pydantic==1.9.0
# !pip install tenacity==8.2.3
# !pip install tiktoken==0.6.0


  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
     ---------------------------------------- 50.9/50.9 kB 1.3 MB/s eta 0:00:00
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-

In [5]:
import sys
import os

# Assuming your PCR_temp folder is in "C:\Users\super\OneDrive\Desktop\research\LawLLM"
directory_path = r'C:\Users\super\OneDrive\Desktop\research\LawLLM\SCR_temp'
if directory_path not in sys.path:
    sys.path.append(directory_path)

In [6]:
from chroma import Chroma
from embeddings import OpenAIEmbeddings
import os
import chromadb
import json
from tqdm import tqdm
import csv
os.environ['OPENAI_API_KEY'] = 'sk-'
import pickle
embd = OpenAIEmbeddings()
client = chromadb.PersistentClient(path="chroma1000")
db = Chroma(client=client, embedding_function=embd, persist_directory='chroma1000')


c:\Users\super\anaconda3\envs\lawllm\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [20]:
import re

def clean_text(text):
    if not isinstance(text, str):
        return text
    # Remove all non-printable characters and excessively problematic symbols
    text = re.sub(r'[^\x20-\x7E]', ' ', text)  # ASCII printable characters
    text = re.sub(r'[\r\n]+', ' ', text)  # Replace newlines and carriage returns with space
    text = re.sub(r'[;"]', ' ', text)  # Remove semicolons and quotes that can break CSV format
    return text.strip()

In [31]:
def format_data(training_cases):
    instruction_key = "### Instruction:"
    instruction_predict = """You are a legal expert who specializes in comparing user-supplied legal cases to a list of candidate legal cases, which includes titles and content. Your main function is to identify and output the title of the most similar case from the list based on the description provided.
You should only output the case title and not any other information.
Consider the following choices:"""
    input_key = "### Input:"
    response_key = "### Response:"
    end_key = "### End"

    choice_list = "\n".join([f"Choice {i}:\n{case}" for i, case in enumerate(training_cases[1:])])
    

    formatted_prompt = (
        f"{instruction_key}\n{instruction_predict}{choice_list}\n"
        f"{input_key}\n"
        f"Case_deatil: {training_cases[0]}\n"
        f"{response_key}\n"
        f"{training_cases[1]}\n"
        f"{end_key}\n"
    )

    return formatted_prompt

    # row_data = [
    #     instruction_key,
    #     instruction_predict,
    #     choice_list,
    #     input_key,
    #     f"Case_detail: {case_json[0]}",
    #     response_key,
    #     case_json[1],
    #     end_key
    # ]
            
                        
    # with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(row_data)

In [39]:
import pandas as pd

data_to_write = []

csv_file_path = r"C:\Users\super\OneDrive\Desktop\research\LawLLM\test_processed2.csv"
json_train_path = r"C:\Users\super\OneDrive\Desktop\research\LawLLM\similar_train.json"

df = pd.read_csv(csv_file_path, header=None)

for index, row in df.iterrows():
    data = row[1]  # Use the actual column name where the data resides

    sections = data.split("###")
    if len(sections) > 2:  # Check if the required section exists
        case_detail = str(sections[2].replace("Input:", '').strip())  # Assuming the case details are in the third section
        training_cases = [str(case_detail)]
        # print(case_detail.strip())

        candidates = db.similarity_search(case_detail, k=10)
        for can in candidates:
            # temp = str(can.page_content).replace('\n', ' ').replace('\r', ' ')
            temp = clean_text(str(can.page_content))
            training_cases.append(str(temp))
            # print(temp)

        train_format = format_data(training_cases)
        print(train_format)

        data_to_write.append(train_format)


        if index > 2:
            break


with open(json_train_path, 'w', encoding='utf-8') as file:
    json.dump(data_to_write, file, ensure_ascii=False, indent=4)

### Instruction:
You are a legal expert who specializes in comparing user-supplied legal cases to a list of candidate legal cases, which includes titles and content. Your main function is to identify and output the title of the most similar case from the list based on the description provided.
You should only output the case title and not any other information.
Consider the following choices:Choice 0:
judges:  (TODD. Circuit Justice, absent). parties:  UNITED STATES v. ANDERSON. opinions:  McNAIRY, District Judge (TODD. Circuit Justice, absent). The first objection taken by the defendant s counsel in this case is that where a man of any age has signed the enlistment, taken the oath, and been mustered in, no judge has a right to interfere by habeas corpus to discharge him until the war department has improperly refused. The constitution of the United States (article 1,   9) declares    that the privilege of the writ of habeas corpus shall not be suspended unless when in eases of rebelli

In [5]:
candidates = db.similarity_search("""Case Title: Patton v. Reily
Case Context: The case involves a dispute over the validity of a deed and whether it could be proved and given in evidence before it had been registered. The judge states that registration was intended by the Legislature to stand in the place of livery of seisin, and that the deed in question cannot be read in evidence.
Case Verdict: Plaintiff win""", k=5)
for can in candidates:
    print(str(can.page_content).replace('\n', ' '))
# print(can.dtype())

judges:   parties:  PATTON et al. v. REILY. opinions:  M’NAIRY, District Judge. The objections •which apply to the first two modes of proof offered by the plaintiffs have been decided as valid by this court upon a former occasion. At that time I was of opinion that the execution of the deed might be proved upon the trial, and given in evidence before it had been registered. After a very full investigation I am constrained to alter that part of my former decision. Registration was intended by the legislature to stand in the place of livery of seizin. By the common law no estate could pass without livery of seizin; and the same may be said as to its substitute. Lands as conveyed by this deed, would not pass the estate at common law; and if it will pass, it must be by act of assembly. The act of 1715 requires the deed to be registered before a legal estate is vested in the grantee. To create a title under this act of assembly the party claiming the benefit of it must have complied with it